In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    gpu = tf.config.experimental.set_memory_growth(gpu, True)
print(len(gpus))

0


In [4]:
df = pd.read_csv('../data/ADANIPORTS.csv')
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123,0.2741


In [5]:
# Date handling
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop = True)
df.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123,0.2741


In [6]:
df.isnull().sum()

Date                    0
Symbol                  0
Series                  0
Prev Close              0
Open                    0
High                    0
Low                     0
Last                    0
Close                   0
VWAP                    0
Volume                  0
Turnover                0
Trades                866
Deliverable Volume      0
%Deliverble             0
dtype: int64

In [7]:
df = df.drop(columns=['Trades','Series','Symbol']) # Un used columns

In [8]:
# Feature engineering daily returns
df['return_1d'] = df['Close'].pct_change()

#Target: up or down next day
df['target'] = (df['return_1d'].shift(-1) > 0).astype(int)

df.head()

,Date,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Deliverable Volume,%Deliverble,return_1d,target
0,2007-11-27,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,9859619,0.3612,NaN,0
1,2007-11-28,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,1453278,0.3172,-0.071659,0
2,2007-11-29,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,1069678,0.2088,-0.010851,1
3,2007-11-30,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,1260913,0.2735,0.042242,1
4,2007-12-03,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,816123,0.2741,0.051815,1


In [9]:
df = df.dropna()
df.isnull().sum()

Date                  0
Prev Close            0
Open                  0
High                  0
Low                   0
Last                  0
Close                 0
VWAP                  0
Volume                0
Turnover              0
Deliverable Volume    0
%Deliverble           0
return_1d             0
target                0
dtype: int64

In [10]:
df["rolling_mean_5"] = df["return_1d"].rolling(5).mean()
df["rolling_std_5"]  = df["return_1d"].rolling(5).std()
df["rolling_mean_10"] = df["return_1d"].rolling(10).mean()
df = df.dropna().reset_index(drop=True)


In [11]:
df.head()                      # label


,Date,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Deliverable Volume,%Deliverble,return_1d,target,rolling_mean_5,rolling_std_5,rolling_mean_10
0,2007-12-11,1075.40,1081.0,1089.0,1041.00,1046.00,1047.65,1067.80,810464,8.654156e+13,415191,0.5123,-0.025804,0,0.001505,0.028223,0.009350
1,2007-12-12,1047.65,1032.0,1065.0,1016.00,1036.90,1036.80,1043.92,744799,7.775137e+13,363848,0.4885,-0.010357,1,-0.008440,0.018700,0.015481
2,2007-12-13,1036.80,1040.0,1150.0,1030.25,1131.15,1129.95,1109.09,3067687,3.402339e+14,1040076,0.3390,0.089844,0,0.009741,0.048351,0.025550
3,2007-12-14,1129.95,1139.9,1140.0,1101.10,1107.00,1110.50,1119.55,1070737,1.198746e+14,525239,0.4905,-0.017213,0,0.002396,0.049276,0.019605
4,2007-12-17,1110.50,1140.0,1168.0,1021.50,1052.00,1044.25,1102.42,1404955,1.548848e+14,670298,0.4771,-0.059658,1,-0.004638,0.056109,0.008457


In [12]:
X = df.drop(columns=["Date", "target"]) # features 
y = df["target"] # label

In [13]:
# --- 1) Chronological train/test split (e.g., last 20% as test)
split = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split], X.iloc[split:]
y_train, y_test = y.iloc[:split], y.iloc[split:]

In [14]:
# --- 2) Two baselines: Logistic Regression & Random Forest
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier


logreg = Pipeline([
    ("scaler", StandardScaler()),                # good for linear models
    ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=3,
    random_state=42,
    n_jobs=-1,
    class_weight="balanced_subsample"
)

# --- 3) Train
logreg.fit(X_train, y_train)
rf.fit(X_train, y_train)

,n_estimators,300
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,3
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [15]:
# --- 4) Evaluate
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix, classification_report

def eval_model(name, model, X_tr, y_tr, X_te, y_te, proba_support=True):
    y_pred_tr = model.predict(X_tr)
    y_pred_te = model.predict(X_te)

    acc_tr = accuracy_score(y_tr, y_pred_tr)
    acc_te = accuracy_score(y_te, y_pred_te)

    p_te, r_te, f1_te, _ = precision_recall_fscore_support(y_te, y_pred_te, average="binary", zero_division=0)
    cm = confusion_matrix(y_te, y_pred_te)

    print(f"\n{name}")
    print("-"*len(name))
    print(f"Train Acc: {acc_tr:.3f} | Test Acc: {acc_te:.3f}")
    print(f"Precision: {p_te:.3f} | Recall: {r_te:.3f} | F1: {f1_te:.3f}")
    print("Confusion Matrix:\n", cm)

    if proba_support:
        # If model can output probabilities, also show ROC-AUC
        try:
            y_proba = model.predict_proba(X_te)[:, 1]
            auc = roc_auc_score(y_te, y_proba)
            print(f"ROC-AUC: {auc:.3f}")
        except Exception:
            pass

# Evaluate both
eval_model("Logistic Regression", logreg, X_train, y_train, X_test, y_test, proba_support=True)
eval_model("Random Forest", rf, X_train, y_train, X_test, y_test, proba_support=True)

# --- 5) Optional: Feature importances for Random Forest
importances = getattr(rf, "feature_importances_", None)
if importances is not None:
    fi = pd.Series(importances, index=X.columns).sort_values(ascending=False)
    print("\nTop 10 RF feature importances:")
    print(fi.head(10))


Logistic Regression
-------------------
Train Acc: 0.522 | Test Acc: 0.498
Precision: 0.532 | Recall: 0.147 | F1: 0.231
Confusion Matrix:
 [[280  44]
 [289  50]]
ROC-AUC: 0.477

Random Forest
-------------
Train Acc: 0.998 | Test Acc: 0.517
Precision: 0.529 | Recall: 0.519 | F1: 0.524
Confusion Matrix:
 [[167 157]
 [163 176]]
ROC-AUC: 0.515

Top 10 RF feature importances:
rolling_std_5         0.084224
rolling_mean_5        0.080887
return_1d             0.080382
rolling_mean_10       0.079690
%Deliverble           0.077710
Deliverable Volume    0.072986
Turnover              0.070596
Volume                0.068989
Prev Close            0.056600
Open                  0.055485
dtype: float64


In [19]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix

# --- clean & align
Xc = X.replace([np.inf, -np.inf], np.nan).dropna()
yc = y.loc[Xc.index]

# --- time-ordered split
split = int(len(Xc) * 0.8)
X_train, X_test = Xc.iloc[:split], Xc.iloc[split:]
y_train, y_test = yc.iloc[:split], yc.iloc[split:]

# --- class imbalance
pos = (y_train == 1).sum()
neg = (y_train == 0).sum()
scale_pos_weight = (neg / pos) if pos > 0 else 1.0

# --- model (note: eval_metric in constructor, not fit)
xgb_clf = XGBClassifier(
    n_estimators=2000,
    max_depth=5,
    learning_rate=0.03,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    objective="binary:logistic",
    random_state=42,
    n_jobs=-1,
    scale_pos_weight=scale_pos_weight,
    tree_method="hist",
    eval_metric="auc"           # <-- moved here
)

# Early stopping via callback (works across xgboost versions 1.x/2.x)
es = EarlyStopping(
    rounds=100,                # patience
    metric_name="auc",
    data_name="validation_1",  # refers to the 2nd tuple in eval_set
    save_best=True
)

xgb_clf.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],

    verbose=False
)

# --- evaluate
y_pred = xgb_clf.predict(X_test)
y_proba = xgb_clf.predict_proba(X_test)[:, 1]

acc  = accuracy_score(y_test, y_pred)
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="binary", zero_division=0)
auc  = roc_auc_score(y_test, y_proba)
cm   = confusion_matrix(y_test, y_pred)

print("XGBoost Results")
print("----------------")
best_iter = getattr(xgb_clf, "best_iteration", None)
print(f"Best trees: {best_iter + 1 if best_iter is not None else 'n/a'}")
print(f"Accuracy : {acc:.3f}")
print(f"Precision: {prec:.3f} | Recall: {rec:.3f} | F1: {f1:.3f}")
print(f"ROC-AUC  : {auc:.3f}")
print("Confusion Matrix:\n", cm)

# --- feature importance
try:
    booster = xgb_clf.get_booster()
    score = booster.get_score(importance_type="gain")
    fmap = {f"f{i}": col for i, col in enumerate(X_train.columns)}
    fi = pd.Series({fmap.get(k, k): v for k, v in score.items()}).sort_values(ascending=False)
except Exception:
    fi = pd.Series(xgb_clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)

print("\nTop 12 feature importances:")
print(fi.head(12).round(6))


XGBoost Results
----------------
Best trees: n/a
Accuracy : 0.528
Precision: 0.538 | Recall: 0.540 | F1: 0.539
ROC-AUC  : 0.525
Confusion Matrix:
 [[167 157]
 [156 183]]

Top 12 feature importances:
Close                 1.577334
VWAP                  1.538829
Low                   1.485206
Turnover              1.423218
Last                  1.400302
rolling_mean_10       1.400136
Deliverable Volume    1.399719
rolling_std_5         1.395866
rolling_mean_5        1.393609
High                  1.386794
%Deliverble           1.385772
return_1d             1.370192
dtype: float64
